In [1]:
# Load the SQL extension first (only shows once)
%load_ext sql

# Import required libraries
import os
from dotenv import load_dotenv
import urllib

# Load variables from your .env file (with UTF-8 encoding)
load_dotenv(encoding='utf-8-sig')

# Fetch variables
server = os.getenv("SERVER")
database = os.getenv("DATABASE")
driver = os.getenv("DRIVER")

# Build the connection string
params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
)
connection_string = f"mssql+pyodbc:///?odbc_connect={params}"

%sql {connection_string}
%config SqlMagic.displaycon = False  # Hide connection info
print("Secure connection established successfully!")

Secure connection established successfully!


Topic: Subquery in SELECT (Derived Column)

Question
Display each employee’s first name, last name, and vacation hours, along with the company-wide average vacation hours.


In [2]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
    PEP.FirstName,
    PEP.LastName,
    HRE.VacationHours,
    (SELECT AVG(VacationHours) FROM HumanResources.Employee) AS CompanyAverageVacationHours
FROM HumanResources.Employee AS HRE
LEFT JOIN Person.Person AS PEP
    ON HRE.BusinessEntityID = PEP.BusinessEntityID;

Done.


FirstName,LastName,VacationHours,CompanyAverageVacationHours
Ken,Sánchez,99,50
Terri,Duffy,1,50
Roberto,Tamburello,2,50


Topic: Subquery in FROM (Derived Table)

Question:
Find the average pay rate per job title, then return only those job titles with an average pay rate greater than 50.

In [3]:
%%sql
SELECT
	JobTitle,
	AveragePayRate
FROM 
	(SELECT
		HRE.[JobTitle],
		AVG(EPH.[Rate]) AS AveragePayRate
	FROM
		HumanResources.Employee HRE
	LEFT JOIN HumanResources.EmployeePayHistory EPH
	ON HRE.BusinessEntityID = EPH.BusinessEntityID
	GROUP BY HRE.[JobTitle]) AS JobTitleSummary
WHERE 
	AveragePayRate > 50;

Done.


JobTitle,AveragePayRate
Chief Executive Officer,125.5000
Information Services Manager,50.4808
Vice President of Engineering,63.4615
Vice President of Production,84.1346
Vice President of Sales,72.1154


Topic: Comparing to a Department (Subquery in WHERE)

Question:
List employees whose vacation hours are greater than the average vacation hours of the Tool Design department


In [4]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
	HRE.[BusinessEntityID],
    PEP.FirstName,
    PEP.LastName,
    HRE.VacationHours
FROM HumanResources.Employee AS HRE
JOIN Person.Person AS PEP
    ON HRE.BusinessEntityID = PEP.BusinessEntityID
WHERE 	HRE.[VacationHours] > 
	(SELECT 
		AVG(E.[VacationHours])
	FROM HumanResources.Employee E
	LEFT JOIN HumanResources.EmployeeDepartmentHistory DH
	ON E.BusinessEntityID = DH.BusinessEntityID
	LEFT JOIN HumanResources.Department D
	ON DH.DepartmentID = D.DepartmentID
	WHERE D.[Name] = 'Tool Design') 

Done.


BusinessEntityID,FirstName,LastName,VacationHours
1,Ken,Sánchez,99
4,Rob,Walters,48
7,Dylan,Miller,61


Topic: Comparing to a Department (Subquery in WHERE)

Question:
List employees (BusinessEntityID, First Name, Last Name, Vacation Hours) whose vacation hours are greater than the average vacation hours of the entire company

In [5]:
%%sql
SELECT  TOP 3 /* Displaying first 3 records to limit output */
	HRE.[BusinessEntityID],
	PEP.[FirstName],
	PEP.[LastName],
	HRE.[VacationHours]
FROM
	HumanResources.Employee HRE
LEFT JOIN
	Person.Person PEP
ON HRE.BusinessEntityID = PEP.BusinessEntityID
WHERE
	HRE.[VacationHours] >
		(SELECT AVG([VacationHours]) FROM HumanResources.Employee)

Done.


BusinessEntityID,FirstName,LastName,VacationHours
1,Ken,Sánchez,99
7,Dylan,Miller,61
8,Diane,Margheim,62


Topic: Correlated Subquery – Per-Row Comparison

Question:
List all products whose ListPrice is greater than the average ListPrice of their respective ProductSubcategory.
Tables: Production.Product, Production.ProductSubcategory

In [6]:
%%sql
SELECT TOP 3  /* Displaying first 3 records to limit output */
    PRP.ProductID,
    PRP.Name AS ProductName,
    PRP.ListPrice,
    PSC.Name AS SubcategoryName,
    (   SELECT 
            AVG(PRP2.ListPrice)
        FROM 
            Production.Product AS PRP2
        WHERE 
            PRP2.ProductSubcategoryID = PRP.ProductSubcategoryID
    ) AS AvgListPriceInSubcategory
FROM Production.Product AS PRP
LEFT JOIN Production.ProductSubcategory AS PSC
    ON PRP.ProductSubcategoryID = PSC.ProductSubcategoryID
WHERE 
    PRP.ListPrice > (
        SELECT 
            AVG(PRP2.ListPrice)
        FROM 
            Production.Product AS PRP2
        WHERE 
            PRP2.ProductSubcategoryID = PRP.ProductSubcategoryID);

Done.


ProductID,ProductName,ListPrice,SubcategoryName,AvgListPriceInSubcategory
771,"Mountain-100 Silver, 38",3399.9900,Mountain Bikes,1683.3650
772,"Mountain-100 Silver, 42",3399.9900,Mountain Bikes,1683.3650
773,"Mountain-100 Silver, 44",3399.9900,Mountain Bikes,1683.3650


Topic: EXISTS – Check for Relationships

Question: List all vendors who have supplied at least one product.
Tables: Purchasing.Vendor, Purchasing.ProductVendor

In [7]:
%%sql
SELECT V.BusinessEntityID, V.Name
FROM Purchasing.Vendor AS V
WHERE EXISTS (
    SELECT 'Vendor Found'
    FROM Purchasing.ProductVendor AS PV
    WHERE PV.BusinessEntityID = V.BusinessEntityID
);

Done.


BusinessEntityID,Name
1492,Australia Bike Retailer
1494,Allenson Cycles
1496,Advanced Bicycles
1498,"Trikes, Inc."
1500,Morgan Bike Accessories
1504,Chicago Rent-All
1506,Greenwood Athletic Company
1508,"Compete Enterprises, Inc"
1510,International
1514,Training Systems


In [8]:
%%sql
--not using EXISTS 

SELECT TOP 3  /* Displaying first 3 records to limit output */
	PV.[BusinessEntityID],
	PV.[Name],
	COUNT(PPV.[ProductID]) AS ProductSold
FROM
	Purchasing.Vendor PV
LEFT JOIN
	Purchasing.ProductVendor PPV
ON PV.BusinessEntityID = PPV.BusinessEntityID
GROUP BY 
	PV.[BusinessEntityID],
	PV.[Name]
HAVING COUNT(PPV.[ProductID]) >= 1
ORDER BY COUNT(PPV.[ProductID]) DESC;

Done.


BusinessEntityID,Name,ProductSold
1656,Mountain Works,30
1540,Bergeron Off-Roads,24
1536,Cruger Bike Company,23


Topic: NOT EXISTS – Find Orphans

Question:
Show all employees who have never been assigned to any department.
Tables: HumanResources.Employee, HumanResources.EmployeeDepartmentHistory

In [9]:
%%sql
SELECT v.BusinessEntityID,
       v.AccountNumber,
       v.Name
FROM Purchasing.Vendor AS v
WHERE NOT EXISTS (
    SELECT 1
    FROM Purchasing.ProductVendor AS pv
    WHERE pv.BusinessEntityID = v.BusinessEntityID
);

Done.


BusinessEntityID,AccountNumber,Name
1502,CYCLING0001,Cycling Master
1512,LIGHTSP0001,Light Speed
1524,RECREATI0001,Recreation Place
1528,IMAGEMA0001,Image Makers Bike Center
1532,KNOPFLER0001,Knopfler Cycles
1550,MERITBI0001,Merit Bikes
1552,SPORTSH0001,Sports House
1558,MARSH0001,Marsh
1564,ILLINOIS0001,Illinois Trek & Clothing
1596,ADATUM0001,A. Datum Corporation


In [10]:
%%sql
--not using NOT EXISTS

SELECT
	PV.[BusinessEntityID],
	PV.[Name],
	COUNT(PPV.[ProductID]) AS ProductSold
FROM
	Purchasing.Vendor PV
LEFT JOIN
	Purchasing.ProductVendor PPV
ON PV.BusinessEntityID = PPV.BusinessEntityID
GROUP BY 
	PV.[BusinessEntityID],
	PV.[Name]
HAVING COUNT(PPV.[ProductID]) = 0
ORDER BY COUNT(PPV.[ProductID]) DESC;

Done.


BusinessEntityID,Name,ProductSold
1502,Cycling Master,0
1512,Light Speed,0
1524,Recreation Place,0
1528,Image Makers Bike Center,0
1532,Knopfler Cycles,0
1550,Merit Bikes,0
1552,Sports House,0
1558,Marsh,0
1564,Illinois Trek & Clothing,0
1596,A. Datum Corporation,0


Topic: IN – Multi-Value Subquery

Question:
List all products that have been ordered in sales orders placed by customers located in the same territories as SalesPersons with a quota history.
Tables: Production.Product, Sales.SalesOrderDetail, Sales.SalesOrderHeader, Sales.SalesPersonQuotaHistory, Sales.SalesPerson